In [102]:
import os
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import torch
import torch.nn as nn
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from sklearn.compose import ColumnTransformer
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
import scipy.stats as stats
from tensorflow.keras.regularizers import l1_l2

In [103]:
big_data = pd.read_csv('/Users/ishan/Desktop/cs229-final-project/big_data.csv')

In [104]:
big_data = big_data.drop(['Unnamed: 0', 'game_id', 'game_date'], axis = 1)


In [105]:
bad_rows = []
for idx, row in big_data.iterrows():
    if idx % 2000 == 0: print(idx)
    if str(row['run_location']) == "nan" and str(row['pass_location']) == "nan":
        bad_rows.append(idx)
big_data.drop(bad_rows, inplace=True)

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000
72000
74000
76000
78000
80000
82000
84000
86000
88000
90000
92000
94000
96000
98000
100000
102000
104000
106000
108000
110000
112000
114000
116000
118000
120000
122000
124000
126000
128000
130000
132000
134000
136000
138000
140000
142000
144000
146000
148000
150000
152000
154000
156000
158000
160000
162000
164000
166000
168000
170000
172000
174000
176000
178000
180000
182000
184000
186000
188000
190000
192000
194000
196000
198000
200000
202000
204000
206000
208000
210000
212000
214000
216000
218000
220000
222000
224000
226000
228000
230000
232000
234000
236000
238000


In [148]:
# cut down number of defense packages
dp_freq = dict(big_data['defense_personnel'].value_counts())
dp_freq[np.NaN] = 0
defense_package = []
for idx, row in big_data.iterrows():
    if dp_freq[row['defense_personnel']] < 450 or row['defense_personnel'] == np.NaN:
        defense_package.append(np.NaN)
    else:
        defense_package.append(row['defense_personnel'])
big_data = big_data.drop(['defense_personnel'],axis=1)
big_data['defense_personnel'] = defense_package

# cut down number of offense packages
op_freq = dict(big_data['offense_personnel'].value_counts())
op_freq[np.NaN] = 0
offense_package = []
for idx, row in big_data.iterrows():
    if op_freq[row['offense_personnel']] < 450 or row['offense_personnel'] == np.NaN:
        offense_package.append(np.NaN)
    else:
        offense_package.append(row['offense_personnel'])
big_data = big_data.drop(['offense_personnel'],axis=1)
big_data['offense_personnel'] = offense_package

In [203]:
teams = ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LV', 'LAC', 'LA', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SF', 'SEA', 'TB', 'TEN', 'WAS']

team_data = big_data
for team in teams:
    relevant = team 
    team_games = team_data[(team_data['home_team'] == relevant) | (team_data['away_team'] == relevant)]
    print(len(team_games), relevant)



13981 ARI
14364 ATL
14376 BAL
14654 BUF
13642 CAR
14482 CIN
13969 CLE
14738 DAL
13608 DEN
13789 DET
14333 GB
14225 HOU
14070 IND
14373 JAX
15903 KC
13884 LV
14091 LAC
14859 LA
13683 MIA
14403 MIN
15070 NE
14446 NO
14216 NYG
13611 NYJ
15022 PHI
14394 PIT
14619 SF
14487 SEA
14969 TB
14385 TEN
13632 WAS


In [178]:
X = team_games.drop(['play_sequence_num','pass_length', 'home_team', 'away_team', 'pass_location', 'run_location', 'play_type', 'surface'], axis=1)

In [179]:
# Building y values 

outcome_run_pass = []
outcome_buckets = []
for idx, row in team_games.iterrows():
    if row['play_type'] == 'pass':
        outcome_run_pass.append('pass')
        outcome_buckets.append('pass + ' + str(row['pass_location']))
    elif row['play_type'] == 'run':
        outcome_run_pass.append('run')
        outcome_buckets.append('run + ' + str(row['run_location']))

In [180]:
X.fillna(0, inplace=True)

In [181]:
len(X)

14619

In [183]:
print(len(Y))

13642


In [184]:
print(outcome_run_pass)

['run', 'pass', 'pass', 'pass', 'run', 'pass', 'pass', 'pass', 'run', 'pass', 'run', 'run', 'pass', 'pass', 'run', 'run', 'pass', 'pass', 'run', 'pass', 'run', 'run', 'pass', 'pass', 'run', 'run', 'pass', 'run', 'pass', 'pass', 'run', 'run', 'run', 'run', 'run', 'pass', 'pass', 'run', 'run', 'run', 'run', 'pass', 'run', 'pass', 'pass', 'pass', 'pass', 'run', 'run', 'run', 'run', 'run', 'pass', 'run', 'run', 'pass', 'run', 'pass', 'run', 'pass', 'pass', 'run', 'pass', 'pass', 'run', 'run', 'pass', 'pass', 'pass', 'run', 'pass', 'pass', 'run', 'run', 'pass', 'pass', 'run', 'pass', 'pass', 'pass', 'pass', 'run', 'run', 'pass', 'run', 'pass', 'run', 'pass', 'run', 'run', 'pass', 'run', 'run', 'pass', 'run', 'pass', 'pass', 'run', 'pass', 'pass', 'pass', 'run', 'pass', 'run', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'run', 'run', 'pass', 'run', 'run', 'run', 'run', 'run', 'pass', 'run', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'run', 'run', 'run', 'run'

In [185]:
X = pd.get_dummies(X)

In [186]:
Y = outcome_run_pass
Y = np.ravel(Y)
len(Y)

14619

In [187]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [188]:
len(X)

14619

In [189]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = to_categorical(encoded_Y)


In [190]:
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, dummy_y, test_size=0.3, random_state=1)

In [191]:
len(X_scaled)

14619

In [192]:
len(dummy_y)

14619

In [193]:
print(len(X.columns))

68


In [198]:
num_samples, num_features = X_scaled.shape
num_timesteps = 9
# This needs to be adjusted based on your actual sequence length

total_elements = X_scaled.size
num_features = len(X.columns)  # Assuming this is fixed based on your dataset
# Find a suitable number of timesteps that allows reshaping
# This is just an illustrative example; in practice, you'd want a meaningful timestep based on your data
for ts in range(1, total_elements):
    if total_elements % (ts * num_features) == 0:
        num_samples = total_elements // (ts * num_features)
        print(f"Possible reshape: ({num_samples}, {ts}, {num_features})")
        break

X_scaled_reshaped = X_scaled.reshape((num_samples, ts, num_features))

# X_scaled_reshaped = X_scaled.reshape((num_samples, num_timesteps, num_features))

model = Sequential([
    LSTM(128, input_shape=(num_timesteps, num_features), return_sequences=False),
    # dropout=0.3,  # Dropout for input units of the layer (keep as comment)
    # recurrent_dropout=0.3,  # Dropout for recurrent units (keep as comment)
    # kernel_regularizer=l1_l2(l1=0.01, l2=0.01),  # L1 and L2 regularization on the kernel (keep as comment)
    # recurrent_regularizer=l1_l2(l1=0.01, l2=0.01),  # L1 and L2 regularization on the recurrent kernel (keep as comment)
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(dummy_y.shape[1], activation='softmax')  # Output layer; using softmax for multi-class classification
])




# model = Sequential([
#     # Adjust the input_shape to match your data's shape: (num_timesteps, num_features)
#     LSTM(128, input_shape=(num_timesteps, num_features), return_sequences=False),
#     dropout=0.2,  # Dropout for input units of the layer
#     recurrent_dropout=0.2,  # Dropout for recurrent units
#     kernel_regularizer=l1_l2(l1=0.01, l2=0.01),  # L1 and L2 regularization on the kernel
#     recurrent_regularizer=l1_l2(l1=0.01, l2=0.01)),
#     Dropout(0.5),
#     Dense(64, activation='relu'),
#     Dense(dummy_y.shape[1], activation='softmax')  # Output layer; using softmax for multi-class classification
# ])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Add early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model
history = model.fit(X_scaled_reshaped, dummy_y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_scaled_reshaped, dummy_y)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

Possible reshape: (14619, 1, 68)


Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_26 (LSTM)                  │ (None, 128)            │       100,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,250 (426.76 KB)

 Trainable params: 109,250 (426.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6783 - loss: 0.6062 - val_accuracy: 0.7442 - val_loss: 0.5341
Epoch 2/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 0s 995us/step - accuracy: 0.7095 - loss: 0.5624 - val_accuracy: 0.7459 - val_loss: 0.5289
Epoch 3/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - accuracy: 0.7190 - loss: 0.5506 - val_accuracy: 0.7462 - val_loss: 0.5318
Epoch 4/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 0s 982us/step - accuracy: 0.7249 - loss: 0.5458 - val_accuracy: 0.7497 - val_loss: 0.5280
Epoch 5/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 0s 987us/step - accuracy: 0.7316 - loss: 0.5342 - val_accuracy: 0.7500 - val_loss: 0.5300
Epoch 6/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7378 - loss: 0.5331 - val_accuracy: 0.7462 - val_loss: 0.5290
Epoch 7/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - accuracy: 0.7230 - loss: 0.5328 - val_accuracy: 0.7435 - val_loss: 0.5357
Epoch 8/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7346 - loss: 0.5256 